In [1]:
import os
import psutil
import json
import shelve
from HelperClass import InvertedIndex, Token
from helper import tokenizeHtml

rootDir = 'D:\\RJ\\UCI\\Ralph School\\2023 Spring\\CS 121\\Assignments\\.vscode\\Res\\Test'
jsonFiles = []
for root, dirs, files in os.walk(rootDir):
    for name in files:
        if name.endswith((".json")):
            full_path = os.path.join(root, name)
            jsonFiles.append(full_path)

jsonFiles = jsonFiles[:5]

In [2]:
def writeDoc(docId:str, url:str):
    with shelve.open(f'DevShelve/Url', 'c') as shelf:
        shelf[str(docId)] = url

def getDocNum():
    with shelve.open(f'DevShelve/Url', 'c') as shelf:
        if 'totalDoc' not in shelf:
            shelf['totalDoc'] = 0
        return int(shelf['totalDoc'])
    
def storeDocNum(totalDoc:int):
    with shelve.open(f'DevShelve/Url', 'c') as shelf:
        shelf['totalDoc'] = totalDoc

def readDocShelve(key):
    with shelve.open(f'DevShelve/Url', 'c') as shelf:
        return shelf[key]

def getDocUrl(docId:int):
    with shelve.open(f'DevShelve/Url', 'c') as shelf:
        if str(docId) in shelf:
            return shelf[str(docId)]
        
# Opening JSON file
def getJsonData(filePath):
    jFile = open(filePath)
    data = json.load(jFile)
    url = data['url']
    htmlContent = data['content']
    jFile.close()
    return (url, htmlContent)

def isValidJsonSize(file):
    fileSize = os.path.getsize(file)/(1024*1024*1024)
    ramUsedGb = psutil.virtual_memory()[3]/1000000000
    totalRam = psutil.virtual_memory().total/(1024*1024*1024)
    if fileSize + ramUsedGb >= (totalRam * .75):
        return False
    return True

def isMemoryFull(limit=75):
    if psutil.virtual_memory()[2] >= limit:
       print(psutil.virtual_memory()[2])
       return True
    return False

def writeData(invIndex, dList, docId):
    invIndex.write('DevShelve')
    invIndex.clear()
    for dItem in dList:
        writeDoc(dItem[0], dItem[1])
    storeDocNum(docId)
    dList.clear()
    print("----Wrote Data to File----")

In [3]:
invIndex = InvertedIndex() #Create inverted index to hold tokens from parser
docId = 0
dList = []
for jFile in jsonFiles:
    docId += 1
    if not isValidJsonSize(jFile):
        writeData(invIndex, dList, docId)

    #Dont Load json file
    if not isValidJsonSize(jFile):
        docId -= 1
    else:
        url, htmlContent = getJsonData(jFile)

        #Check if already parsed
        if url == getDocUrl(docId):
            print(f'Parsed Already -- {docId}:{url}')
        else:
            # Cleans and parses HTML content into tokens then adds it to Inverted index
            tokenizeHtml(docId=docId, invIndex=invIndex, htmlContent=htmlContent)
            dList.append((docId,url))
            print(docId, url)

            if isMemoryFull():
                print('memFull')
                writeData(invIndex, dList, docId)

if docId != getDocNum():
    writeData(invIndex, dList, docId)

1 https://www.ics.uci.edu/~irani/s15-6D/ClassNotes/23_GeneratingPermsSubsets.html
2 https://grape.ics.uci.edu/wiki/public/wiki/cs222-2019-fall-git?version=4
3 http://archive.ics.uci.edu/ml/datasets/Gas+sensor+array+exposed+to+turbulent+gas+mixtures
4 http://alderis.ics.uci.edu/links.html
5 https://aiclub.ics.uci.edu/
----Wrote Data to File----


In [6]:
invIndex

Total Tokens: 0